In [46]:
import pandas as pd
import numpy as np

from pgmpy.estimators import BicScore, K2Score, HillClimbSearch, BDeuScore, BayesianEstimator
from pgmpy.models import BayesianModel, BayesianNetwork, NaiveBayes

In [47]:
data = pd.read_csv("Customer-Churn-Records.csv")

In [48]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,cardtype,gender,geography
0,1,15634602,Hargrave,619,France,42,2,0.00,1,1,1,101348.88,1,1,2,464,4,1,1
1,2,15647311,Hill,608,Spain,41,1,83807.86,1,0,1,112542.58,0,1,3,456,4,1,2
2,3,15619304,Onio,502,France,42,8,159660.80,3,1,0,113931.57,1,1,3,377,4,1,1
3,4,15701354,Boni,699,France,39,1,0.00,2,0,0,93826.63,0,0,5,350,2,1,1
4,5,15737888,Mitchell,850,Spain,43,2,125510.82,1,1,1,79084.10,0,0,5,425,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,39,5,0.00,2,1,0,96270.64,0,0,1,300,4,0,1
9996,9997,15569892,Johnstone,516,France,35,10,57369.61,1,1,1,101699.77,0,0,5,771,3,0,1
9997,9998,15584532,Liu,709,France,36,7,0.00,1,0,1,42085.58,1,1,3,564,1,1,1
9998,9999,15682355,Sabbatini,772,Germany,42,3,75075.31,2,1,0,92888.52,1,1,2,339,2,0,3


In [49]:
def round_1(number, multiple_of = 10):
    number = round(number/multiple_of)
    number *= multiple_of
    return int(number)

In [50]:
balance_func = lambda x: round_1(x, 10000)
data['Balance'] = data['Balance'].map(balance_func)

salary_func = lambda x: round_1(x, 1000)
data['EstimatedSalary'] = data['EstimatedSalary'].map(salary_func)

credit_func = lambda x: round_1(x, 10)
data['CreditScore'] = data['CreditScore'].map(credit_func)

point_func = lambda x: round_1(x, 100)
data['Point Earned'] = data['Point Earned'].map(point_func)

age_func = lambda x: round_1(x, 5)
data['Age'] = data['Age'].map(age_func)

In [51]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,cardtype,gender,geography
0,1,15634602,Hargrave,620,France,40,2,0,1,1,1,101000,1,1,2,500,4,1,1
1,2,15647311,Hill,610,Spain,40,1,80000,1,0,1,113000,0,1,3,500,4,1,2
2,3,15619304,Onio,500,France,40,8,160000,3,1,0,114000,1,1,3,400,4,1,1
3,4,15701354,Boni,700,France,40,1,0,2,0,0,94000,0,0,5,400,2,1,1
4,5,15737888,Mitchell,850,Spain,45,2,130000,1,1,1,79000,0,0,5,400,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,770,France,40,5,0,2,1,0,96000,0,0,1,300,4,0,1
9996,9997,15569892,Johnstone,520,France,35,10,60000,1,1,1,102000,0,0,5,800,3,0,1
9997,9998,15584532,Liu,710,France,35,7,0,1,0,1,42000,1,1,3,600,1,1,1
9998,9999,15682355,Sabbatini,770,Germany,40,3,80000,2,1,0,93000,1,1,2,300,2,0,3


In [52]:
data = data.drop(columns = ['RowNumber', 'CustomerId', 'Surname', 'Geography'])

In [53]:
data

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,cardtype,gender,geography
0,620,40,2,0,1,1,1,101000,1,1,2,500,4,1,1
1,610,40,1,80000,1,0,1,113000,0,1,3,500,4,1,2
2,500,40,8,160000,3,1,0,114000,1,1,3,400,4,1,1
3,700,40,1,0,2,0,0,94000,0,0,5,400,2,1,1
4,850,45,2,130000,1,1,1,79000,0,0,5,400,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,770,40,5,0,2,1,0,96000,0,0,1,300,4,0,1
9996,520,35,10,60000,1,1,1,102000,0,0,5,800,3,0,1
9997,710,35,7,0,1,0,1,42000,1,1,3,600,1,1,1
9998,770,40,3,80000,2,1,0,93000,1,1,2,300,2,0,3


In [54]:
hc = HillClimbSearch(data)
best_model = hc.estimate(scoring_method = K2Score(data))
best_model.edges()

  0%|          | 0/1000000 [00:00<?, ?it/s]

OutEdgeView([('Age', 'IsActiveMember'), ('Balance', 'geography'), ('Balance', 'Exited'), ('NumOfProducts', 'Exited'), ('NumOfProducts', 'Balance'), ('NumOfProducts', 'geography'), ('Exited', 'Complain'), ('Exited', 'Age'), ('Exited', 'geography'), ('Exited', 'IsActiveMember'), ('Complain', 'gender')])

In [39]:
# best_model.get_independencies()

In [55]:
model = BayesianNetwork([('Age', 'IsActiveMember'), ('Balance', 'geography'), ('Balance', 'Exited'), ('NumOfProducts', 'Exited'), ('NumOfProducts', 'Balance'), ('NumOfProducts', 'geography'), ('Complain', 'Exited'), ('Age', 'Exited'), ('geography', 'Exited'), ('IsActiveMember', 'Exited'), ('Complain', 'gender')])

In [56]:
from pgmpy.factors.discrete.CPD import TabularCPD

def print_full(cpd):
    backup = TabularCPD._truncate_strtable
    TabularCPD._truncate_strtable = lambda self, x: x
    print(cpd)
    TabularCPD._truncate_strtable = backup

In [57]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
test

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,cardtype,gender,geography
7142,610,65,2,130000,2,1,0,122000,1,1,4,600,1,1,2
9805,640,35,7,170000,1,0,1,44000,0,0,1,900,4,0,1
646,750,30,3,0,2,1,1,105000,0,0,4,1000,1,1,1
6385,610,30,1,110000,2,0,1,59000,0,0,3,500,2,1,3
5274,620,45,8,0,2,1,0,101000,0,0,2,600,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,590,40,5,140000,2,1,0,159000,0,0,5,400,1,0,3
7333,700,45,2,140000,2,1,0,83000,0,0,4,200,3,0,2
9718,710,60,3,130000,2,1,0,119000,1,1,2,500,3,0,2
7414,740,45,2,100000,2,1,1,79000,0,0,5,900,2,1,3


In [59]:
model.fit(train)

predict_data = test.copy()
predict_data = predict_data.drop(columns = ['Exited', 'CreditScore', 'Tenure', 'HasCrCard', 'Satisfaction Score', 'Point Earned', 'cardtype', 'EstimatedSalary'])
y_pred = model.predict(predict_data)
y_pred

  0%|          | 0/1151 [00:00<?, ?it/s]

,Exited
0,0
1,0
2,0
3,0
4,0
...,...
1995,0
1996,0
1997,1
1998,0


In [60]:
test['Exited']

7142    1
9805    0
646     0
6385    0
5274    0
       ..
556     0
7333    0
9718    1
7414    0
8361    0
Name: Exited, Length: 2000, dtype: int64

In [61]:
truth = test['Exited'].copy()
from sklearn.metrics import classification_report
print(classification_report(truth, y_pred['Exited'].copy()))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1605
           1       0.99      0.69      0.81       395

    accuracy                           0.94      2000
   macro avg       0.96      0.84      0.89      2000
weighted avg       0.94      0.94      0.93      2000

